In [ ]:
from PIL import Image
import numpy as np
import numpy.linalg 
import re
import sys, os
import functools
import glob
numpy.set_printoptions(precision=3)
# Data
X = np.array([[0., 0., 1.],[1.,0.,0.], [2.,2.,2.], [2.,5.,4.]])
Y = np.array([[0.1, -0.2], [0.9, 1.1], [6.2, 5.9], [11.9, 12.3]])

#Testing
PLS(X,Y)



In [ ]:
# centralize the data

def centralize(X,Y):
    X=X-np.mean(X)
    Y=Y-np.mean(Y)
    return (X,Y)

def PLS(X,Y,tol=1e-6):

    N = X.shape[0]
    u = numpy.random.rand(1,N)
    u = numpy.asmatrix(u).T
    
    #initialize T，U,W,C
    T=U=W=C=None 

    #initialize P,Q,Bdiag
    P=Q=Bdiag=None

    i=0
    while(True):                             
        j = 0
        while (True):
            #Step(1)
            w = X.T * u   
            #Step(2) Normalizes the magnitude of w to 1.
            w = w / numpy.linalg.norm(w)
            #Step(3) a new approximation to t 
            t = X * w  
            t = t / numpy.linalg.norm(t) #Normalize t
            #Step(4) Obtain c
            c = Y.T * t
            #Step(5) Normalize c 
            c = c / numpy.linalg.norm(c)        
            #Step(6) Store the currently used value for u
            u_old = u
            #Step(7) A new candidate for the latent vector for Y. 
            u = Y * c
            #Step(8) Terminate the iterations if error is small enough
            error = numpy.linalg.norm(u - u_old)
            if error < tol:
                break

            j += 1 

        #Obtain matrix b
        b = t.T * u
        b = b[0,0]
        
        if T is None:
            T = t
        else:
            T = numpy.hstack((T,t))
        if U is None:
            U = u
        else:
            U = numpy.hstack((U,u))
        if W is None:
            W = w
        else:
            W = numpy.hstack((W,w))
        if C is None:
            C = c
        else:
            C = numpy.hstack((C,c))
     
        p = X.T * t / (np.linalg.norm(t) ** 2)
        q = Y.T * u / (np.linalg.norm(u) ** 2)

        if P is None:
            P = p
        else:
            P = numpy.hstack((P,p))
        if Q is None:
            Q = q
        else:
            Q = numpy.hstack((Q,q))
        if Bdiag is None:
            Bdiag = [b]
        else:
            Bdiag.append(b)
         
        X_old = X
        Y_old = Y
        X = X - t * p.T
        Y = Y - b * t * c.T
        i += 1

        if numpy.linalg.norm(X) < 0.001: break

        #Obtain B
        B = numpy.diag(Bdiag)
        B = numpy.asmatrix(B) 
        B = numpy.linalg.pinv(P.T) * B * C.T
        
        #Prediction based on the original X:
        
        Y_predicted = centralize(X,Y)[0] * B
        print("\nY_predicted from the original X:")
        print(Y_predicted)

        Y_predicted_with_mean = Y_predicted + np.mean(Y)
        print("\nThe predicted Y with the original Y's column-wise mean added:")
        print(Y_predicted_with_mean)
         
        return B

# Data
X = np.array([[0., 0., 1.],[1.,0.,0.], [2.,2.,2.], [2.,5.,4.]])
Y = np.array([[0.1, -0.2], [0.9, 1.1], [6.2, 5.9], [11.9, 12.3]])

#Testing
PLS(X,Y)



In [ ]:
# Data
X = np.array([[0., 0., 1.],[1.,0.,0.], [2.,2.,2.], [2.,5.,4.]])
Y = np.array([[0.1, -0.2], [0.9, 1.1], [6.2, 5.9], [11.9, 12.3]])

#Testing
PLS(X,Y)



In [ ]:
#PLS1

def PLS1(X,Y,tol=1e-6):
    if Y.shape[1] != 1:
        raise ValueError("PLS1 can only be called when the Y has only one column")
    X,Y=centralize(X,Y)
    T=U=W=C=P=Q=B=t=w=u=c=p=q=None        
    u = Y
    i = 0
    while (True):
        w = np.dot(X.T , u)
        w = w / numpy.linalg.norm(w)
        t = np.dot(X , w)
        c = np.dot(Y.T , t)
        c = c / numpy.linalg.norm(c)        
        u = np.dot(Y , c)
        if T is None:
            T = t
        else:
            T = numpy.hstack((T,t))
        if U is None:
            U = u
        else:
            U = numpy.hstack((U,u))
        if W is None:
            W = w
        else:
            W = numpy.hstack((W,w))
        p = np.dot(X.T,t/(numpy.linalg.norm(t)**2))
        q = np.dot(Y.T,u/(numpy.linalg.norm(u)**2))

        if P is None:
            P = p
        else:
            P = numpy.hstack((P,p))
        if Q is None:
            Q = q
        else:
            Q = numpy.hstack((Q,q))

        X_old = X
        Y_old = Y
        xdot=np.dot(t, p.T)
        X = X - xdot
        Y = Y - np.dot(xdot,Y)/(np.linalg.norm(t)**2)
        i += 1
        if np.linalg.norm(X) < 0.001: break
         
        B = W * ((P.T * W).I) * T.T * Y
       
        return B


